In [6]:
import hwi_igraph_testing_main as ig
import importlib
import descriptors as ds
import os  
import time
import matplotlib.pyplot as plt
import tracemalloc

importlib.reload(ig)  # 강제 리로드

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

targetFileName = '0.25'
loop_cnt = 1

    
for test_file in test_files:
    if targetFileName not in test_file:
        continue

    import time
    total_graph_time = 0
    for i in range(loop_cnt):

        # g = ig.generateGraph(data_path + test_file + ".txt")
        tracemalloc.start()
        graph_start = time.time()
        g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
        _stats = tracemalloc.get_traced_memory()
        graph_end = time.time()     
        tracemalloc.stop()
        graph_mem = _stats[1]-_stats[0]  
        stats = ds.desciptors(g)
        total_graph_time += graph_end - graph_start
        #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    descriptor_time = stats["time"]
    descriptor_mem = stats["mem"]

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = total_graph_time/loop_cnt
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



new method Green Edges len :  1982
previous method Green vertex neighbors:  [13, 14, 14, 15, 27, 28, 28, 29, 46, 46, 47, 47, 55, 56, 56, 57, 63, 64, 64, 65, 82, 83, 83, 84, 84, 85, 97, 98, 98, 99, 99, 100, 115, 116, 116, 117, 117, 124, 125, 125, 126, 126, 127, 133, 134, 134, 135, 135, 148, 149, 149, 150, 150, 151, 151, 152, 152, 153, 153, 154, 168, 169, 169, 170, 170, 171, 184, 185, 185, 186, 186, 187, 193, 194, 194, 195, 195, 196, 204, 204, 205, 205, 206, 211, 212, 212, 213, 213, 214, 214, 215, 215, 216, 216, 217, 217, 218, 218, 219, 219, 220, 220, 221, 221, 222, 222, 223, 239, 240, 240, 241, 241, 242, 252, 253, 253, 254, 254, 255, 255, 256, 262, 263, 263, 264, 264, 265, 274, 275, 275, 276, 276, 277, 277, 278, 280, 280, 281, 281, 282, 282, 283, 283, 284, 284, 285, 285, 286, 286, 287, 287, 288, 288, 289, 310, 311, 311, 312, 312, 313, 321, 322, 322, 323, 323, 324, 324, 325, 331, 332, 332, 333, 333, 334, 345, 346, 346, 347, 347, 348, 348, 350, 350, 351, 351, 352, 368, 369, 369, 370, 370,

In [7]:
import hwi_igraph_testing_main as ig
import igraph_testing_main as ig2
import importlib
import descriptors as ds
import os  
import time
import matplotlib.pyplot as plt
import tracemalloc

importlib.reload(ig)  # 강제 리로드

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

targetFileName = '100-2D'
loop_cnt = 1

    
for test_file in test_files:
    if targetFileName not in test_file:
        continue

    import time
    total_graph_time = 0
    for i in range(loop_cnt):

        # g = ig.generateGraph(data_path + test_file + ".txt")
        tracemalloc.start()
        graph_start = time.time()
        g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
        _stats = tracemalloc.get_traced_memory()
        graph_end = time.time()     
        tracemalloc.stop()
        graph_mem = _stats[1]-_stats[0]  
        stats = ds.desciptors(g)
        total_graph_time += graph_end - graph_start
        #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    descriptor_time = stats["time"]
    descriptor_mem = stats["mem"]

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = total_graph_time/loop_cnt
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



new method Green Edges len :  400
previous method Green vertex neighbors:  [4900, 4900, 4901, 4901, 4902, 4902, 4903, 4903, 4904, 4904, 4905, 4905, 4906, 4906, 4907, 4907, 4908, 4908, 4909, 4909, 4910, 4910, 4911, 4911, 4912, 4912, 4913, 4913, 4914, 4914, 4915, 4915, 4916, 4916, 4917, 4917, 4918, 4918, 4919, 4919, 4920, 4920, 4921, 4921, 4922, 4922, 4923, 4923, 4924, 4924, 4925, 4925, 4926, 4926, 4927, 4927, 4928, 4928, 4929, 4929, 4930, 4930, 4931, 4931, 4932, 4932, 4933, 4933, 4934, 4934, 4935, 4935, 4936, 4936, 4937, 4937, 4938, 4938, 4939, 4939, 4940, 4940, 4941, 4941, 4942, 4942, 4943, 4943, 4944, 4944, 4945, 4945, 4946, 4946, 4947, 4947, 4948, 4948, 4949, 4949, 4950, 4950, 4951, 4951, 4952, 4952, 4953, 4953, 4954, 4954, 4955, 4955, 4956, 4956, 4957, 4957, 4958, 4958, 4959, 4959, 4960, 4960, 4961, 4961, 4962, 4962, 4963, 4963, 4964, 4964, 4965, 4965, 4966, 4966, 4967, 4967, 4968, 4968, 4969, 4969, 4970, 4970, 4971, 4971, 4972, 4972, 4973, 4973, 4974, 4974, 4975, 4975, 4976, 4976, 